#Beam Element - This notebook computes the mass matrix and internal forces and outputs them for later use

1. 1 rotation 3 displacements (Reduced dimensions so all matricies are subsequently invertible, Mii = 0 if some dof's are accounted for but  unused)
2. Interpolation of roataion matricies WITHOUT orthogonalization (sympy struggling with too much algebra)
3. Circular cross-section with radius 'r'

In [1]:
import numpy as np
import scipy as sp
import sympy as sym
import pickle

from scipy import linalg
from sympy import mpmath
from sympy import cos, sin

from IPython.display import display
from __future__ import division
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

#### Define Needed Functions

In [2]:
def Skew_sym(v):
    """
    This function returns the skew symetric matrix 
    of the vector 'v' to affect the cross product of 'v'x'u'
    """
    v_matrix = sym.Matrix([[  0 , -v[2],  v[1]],
                          [v[2],     0,  -v[0]],
                          [-v[1],  v[0],    0]])
    return v_matrix

In [3]:
def Axial_sym(A):
    a_vec = 1/2*sym.Matrix([A[2,1] - A[1,2], A[0,2] - A[2,0], A[1,0] - A[0,1]])
    return a_vec

In [4]:
def Rotate_sym(theta):
    """
    This function returns the symbolic rotation matrix 
    for the simple 2D-rotation about the third axis
    """
    R = sym.Matrix([[cos(theta),-sin(theta), 0],
                    [sin(theta), cos(theta), 0],[0,0,1]])
    return R

#### Define symbolic quantites

In [5]:
# nodal positons, velocities, and accelerations
"""
    q = [u_1x,u_1y,u_1z,theta_1,u_2x,u_2y,u_2z,theta_2]
    q = 1x6 for a 2D beam with 2 nodes (neglecting u_iz)
"""

# symbolic system parameters 
E, I, A, rho, x, l, r, tau_a = sym.symbols('E I A rho x l r tau_a')

# Define radius consistent with Area used in simulation
# radius = 0.02393653682

# Beam coordinates at cross-section 1 and 2
# point 'p' is the centroid
r1 = sym.Matrix(['r_x','r_y'])
r2 = sym.Matrix(['r_x','r_y'])
# r1 = sym.Matrix(['0','0'])
# r2 = sym.Matrix(['0','0'])

# locating the point in the cross section
s1 = sym.Matrix([r1, [0]])
s2 = sym.Matrix([r2, [0]])
s = sym.Matrix.vstack(s1,s2)

# generalized coordinates
q = sym.Matrix(sym.symarray('q',8))
# generalized speeds
qdot = sym.Matrix(sym.symarray('qdot',len(q)))
# generalized accelerations
qddot = sym.Matrix(sym.symarray('qddot',len(q)))

# Deformations of Nodes  
u = sym.Matrix([q[1:4,0], q[5:8,0]])
udot = sym.Matrix([qdot[1:4,0], qdot[5:8,0]])
uddot = sym.Matrix([qddot[1:4,0], qddot[5:8,0]])

display([q,qdot,qddot])
display([u,udot,uddot])

⎡⎡q₀⎤, ⎡q̇₀⎤, ⎡q̈₀⎤⎤
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₁⎥  ⎢q̇₁⎥  ⎢q̈₁⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₂⎥  ⎢q̇₂⎥  ⎢q̈₂⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₃⎥  ⎢q̇₃⎥  ⎢q̈₃⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₄⎥  ⎢q̇₄⎥  ⎢q̈₄⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₅⎥  ⎢q̇₅⎥  ⎢q̈₅⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₆⎥  ⎢q̇₆⎥  ⎢q̈₆⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎣⎣q₇⎦  ⎣q̇₇⎦  ⎣q̈₇⎦⎦

⎡⎡q₁⎤, ⎡q̇₁⎤, ⎡q̈₁⎤⎤
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₂⎥  ⎢q̇₂⎥  ⎢q̈₂⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₃⎥  ⎢q̇₃⎥  ⎢q̈₃⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₅⎥  ⎢q̇₅⎥  ⎢q̈₅⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎢⎢q₆⎥  ⎢q̇₆⎥  ⎢q̈₆⎥⎥
⎢⎢  ⎥  ⎢   ⎥  ⎢   ⎥⎥
⎣⎣q₇⎦  ⎣q̇₇⎦  ⎣q̈₇⎦⎦

### Needed Matrix Quantities

In [6]:
# angular velocity for 2D case is trivial
omega1_skew = Skew_sym([0,0,qdot[0]])
omega2_skew = Skew_sym([0,0,qdot[4]])
alpha1_skew = Skew_sym([0,0,qddot[0]])
alpha2_skew = Skew_sym([0,0,qddot[4]])

# 1D Roation at each node
R1 = Rotate_sym(q[0])
R2 = Rotate_sym(q[4])
# display(R1,R2)

# "spatial" rotation matrix
R = sym.Matrix.vstack(sym.Matrix.hstack(R1,sym.zeros(3)), \
                      sym.Matrix.hstack(sym.zeros(3),R2))

# "spatial" angular velocity matrix
Omega_skew = sym.Matrix.vstack(sym.Matrix.hstack(omega1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),omega2_skew))

# "spatial" angular acceleration matrix
Alpha_skew = sym.Matrix.vstack(sym.Matrix.hstack(alpha1_skew,sym.zeros(3)), \
                               sym.Matrix.hstack(sym.zeros(3),alpha2_skew))
# display(Omega)

In [7]:
display(R1*omega1_skew)

⎡-q̇₀⋅sin(q₀)  -q̇₀⋅cos(q₀)  0⎤
⎢                             ⎥
⎢q̇₀⋅cos(q₀)   -q̇₀⋅sin(q₀)  0⎥
⎢                             ⎥
⎣     0             0        0⎦

In [8]:
display(s1)

⎡rₓ ⎤
⎢   ⎥
⎢r_y⎥
⎢   ⎥
⎣ 0 ⎦

### Define Kinematics

In [9]:
# Define velocity of element endpoints (nodes)
v = sym.simplify(udot + R*Omega_skew*s)
print('v = ')
display(v)

# Define acceleration of element endpoints (nodes)
a = sym.simplify(uddot + R*Omega_skew*Omega_skew*s + R*Alpha_skew*s)
print('\na = ')
display(a)

v = 


⎡-q̇₀⋅rₓ⋅sin(q₀) - q̇₀⋅r_y⋅cos(q₀) + q̇₁⎤
⎢                                       ⎥
⎢q̇₀⋅rₓ⋅cos(q₀) - q̇₀⋅r_y⋅sin(q₀) + q̇₂ ⎥
⎢                                       ⎥
⎢                  q̇₃                  ⎥
⎢                                       ⎥
⎢-q̇₄⋅rₓ⋅sin(q₄) - q̇₄⋅r_y⋅cos(q₄) + q̇₅⎥
⎢                                       ⎥
⎢q̇₄⋅rₓ⋅cos(q₄) - q̇₄⋅r_y⋅sin(q₄) + q̇₆ ⎥
⎢                                       ⎥
⎣                  q̇₇                  ⎦


a = 


⎡                                             2                 2            ⎤
⎢-q̈₀⋅rₓ⋅sin(q₀) - q̈₀⋅r_y⋅cos(q₀) + q̈₁ - q̇₀ ⋅rₓ⋅cos(q₀) + q̇₀ ⋅r_y⋅sin(q₀)⎥
⎢                                                                            ⎥
⎢                                            2                 2             ⎥
⎢q̈₀⋅rₓ⋅cos(q₀) - q̈₀⋅r_y⋅sin(q₀) + q̈₂ - q̇₀ ⋅rₓ⋅sin(q₀) - q̇₀ ⋅r_y⋅cos(q₀) ⎥
⎢                                                                            ⎥
⎢                                    q̈₃                                     ⎥
⎢                                                                            ⎥
⎢                                             2                 2            ⎥
⎢-q̈₄⋅rₓ⋅sin(q₄) - q̈₄⋅r_y⋅cos(q₄) + q̈₅ - q̇₄ ⋅rₓ⋅cos(q₄) + q̇₄ ⋅r_y⋅sin(q₄)⎥
⎢                                                                            ⎥
⎢                                            2                 2             ⎥
⎢q̈₄⋅rₓ⋅cos(q₄) - q̈₄⋅r_y⋅sin(q₄) + q̈₆ - q̇₄ ⋅rₓ⋅si

### Compute the Mass Matrix

In [10]:
# Define shape function for element with one node at each end
h = sym.symarray('h', 2)

h[0] = sym.Rational(1,2)*(1 - x)
h[1] = sym.Rational(1,2)*(1 + x)

# Compute shape function matrix
H = sym.expand(sym.Matrix([h[0]*sym.eye(3), h[1]*sym.eye(3)])).T
print('\nH = ')
display(H)

# Define velocity of any point 
Vp = H*v
print('\nV = ')
display(Vp)

# Define velocity of any point 
Ap = H*a
# print('\nA = ')
# display(Accel)

# Compute partial velocities of the nodes
Vr = sym.Matrix([[sym.diff(Vp,qdot) for Vp in Vp] for qdot in qdot]).T
# v_r = H
print('\nVr = ')
display(Vr)
# print(Vr.shape)

# Compute mass matrix
M = sym.factor(sym.Matrix([[sym.expand(sym.integrate(Vr[:,i].dot(Ap)*rho,('r_x',0,r),('r_y',0,r),(x,0,l))).coeff(qddot[j]) \
                            for i in range(len(qddot))] for j in range(len(qddot))]))

pickle.dump( M, open( "gebf-mass-matrix.dump", "wb" ) )


H = 


⎡  x   1                    x   1              ⎤
⎢- ─ + ─     0        0     ─ + ─    0      0  ⎥
⎢  2   2                    2   2              ⎥
⎢                                              ⎥
⎢           x   1                  x   1       ⎥
⎢   0     - ─ + ─     0       0    ─ + ─    0  ⎥
⎢           2   2                  2   2       ⎥
⎢                                              ⎥
⎢                    x   1                x   1⎥
⎢   0        0     - ─ + ─    0      0    ─ + ─⎥
⎣                    2   2                2   2⎦


V = 


⎡⎛  x   1⎞                                             ⎛x   1⎞                
⎢⎜- ─ + ─⎟⋅(-q̇₀⋅rₓ⋅sin(q₀) - q̇₀⋅r_y⋅cos(q₀) + q̇₁) + ⎜─ + ─⎟⋅(-q̇₄⋅rₓ⋅sin(q₄
⎢⎝  2   2⎠                                             ⎝2   2⎠                
⎢                                                                             
⎢ ⎛  x   1⎞                                            ⎛x   1⎞                
⎢ ⎜- ─ + ─⎟⋅(q̇₀⋅rₓ⋅cos(q₀) - q̇₀⋅r_y⋅sin(q₀) + q̇₂) + ⎜─ + ─⎟⋅(q̇₄⋅rₓ⋅cos(q₄)
⎢ ⎝  2   2⎠                                            ⎝2   2⎠                
⎢                                                                             
⎢                                          ⎛  x   1⎞       ⎛x   1⎞            
⎢                                      q̇₃⋅⎜- ─ + ─⎟ + q̇₇⋅⎜─ + ─⎟            
⎣                                          ⎝  2   2⎠       ⎝2   2⎠            

                          ⎤
) - q̇₄⋅r_y⋅cos(q₄) + q̇₅)⎥
                          ⎥
                          ⎥
                  


Vr = 


⎡⎛  x   1⎞                                x   1                    ⎛x   1⎞    
⎢⎜- ─ + ─⎟⋅(-rₓ⋅sin(q₀) - r_y⋅cos(q₀))  - ─ + ─     0        0     ⎜─ + ─⎟⋅(-r
⎢⎝  2   2⎠                                2   2                    ⎝2   2⎠    
⎢                                                                             
⎢⎛  x   1⎞                                         x   1           ⎛x   1⎞    
⎢⎜- ─ + ─⎟⋅(rₓ⋅cos(q₀) - r_y⋅sin(q₀))      0     - ─ + ─     0     ⎜─ + ─⎟⋅(rₓ
⎢⎝  2   2⎠                                         2   2           ⎝2   2⎠    
⎢                                                                             
⎢                                                           x   1             
⎢                  0                       0        0     - ─ + ─             
⎣                                                           2   2             

                          x   1              ⎤
ₓ⋅sin(q₄) - r_y⋅cos(q₄))  ─ + ─    0      0  ⎥
                          2   2     

In [11]:
# print('\nM = \n')
# display(M)

print('M_11 = ')
display(M[0:4,0:4])
print('\nM_22 = ')
display(M[4:8,4:8])
print('\nM_12 = ')
display(M[0:4,4:8])
print('\nM_21.T = ')
display(M[4:8,0:4].T)

M_11 = 


⎡   4   ⎛   2          2    ⎞ ⎛ 2          ⎞      3                       ⎛ 2 
⎢l⋅r ⋅ρ⋅⎝sin (q₀) + cos (q₀)⎠⋅⎝l  - 3⋅l + 3⎠  -l⋅r ⋅ρ⋅(sin(q₀) + cos(q₀))⋅⎝l  
⎢───────────────────────────────────────────  ────────────────────────────────
⎢                     18                                           24         
⎢                                                                             
⎢    3                       ⎛ 2          ⎞                 2   ⎛ 2          ⎞
⎢-l⋅r ⋅ρ⋅(sin(q₀) + cos(q₀))⋅⎝l  - 3⋅l + 3⎠              l⋅r ⋅ρ⋅⎝l  - 3⋅l + 3⎠
⎢───────────────────────────────────────────             ─────────────────────
⎢                     24                                           12         
⎢                                                                             
⎢   3                        ⎛ 2          ⎞                                   
⎢l⋅r ⋅ρ⋅(-sin(q₀) + cos(q₀))⋅⎝l  - 3⋅l + 3⎠                                   
⎢──────────────────────────────────────────         


M_22 = 


⎡   4   ⎛   2          2    ⎞ ⎛ 2          ⎞      3                       ⎛ 2 
⎢l⋅r ⋅ρ⋅⎝sin (q₄) + cos (q₄)⎠⋅⎝l  + 3⋅l + 3⎠  -l⋅r ⋅ρ⋅(sin(q₄) + cos(q₄))⋅⎝l  
⎢───────────────────────────────────────────  ────────────────────────────────
⎢                     18                                           24         
⎢                                                                             
⎢    3                       ⎛ 2          ⎞                 2   ⎛ 2          ⎞
⎢-l⋅r ⋅ρ⋅(sin(q₄) + cos(q₄))⋅⎝l  + 3⋅l + 3⎠              l⋅r ⋅ρ⋅⎝l  + 3⋅l + 3⎠
⎢───────────────────────────────────────────             ─────────────────────
⎢                     24                                           12         
⎢                                                                             
⎢   3                        ⎛ 2          ⎞                                   
⎢l⋅r ⋅ρ⋅(-sin(q₄) + cos(q₄))⋅⎝l  + 3⋅l + 3⎠                                   
⎢──────────────────────────────────────────         


M_12 = 


⎡    4   ⎛ 2    ⎞                                          3   ⎛ 2    ⎞       
⎢-l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₀)⋅sin(q₄) + cos(q₀)⋅cos(q₄))   l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q
⎢─────────────────────────────────────────────────────  ──────────────────────
⎢                          18                                            24   
⎢                                                                             
⎢            3   ⎛ 2    ⎞                                            2   ⎛ 2  
⎢         l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₄) + cos(q₄))                    -l⋅r ⋅ρ⋅⎝l  -
⎢         ───────────────────────────────────                    ─────────────
⎢                          24                                            12   
⎢                                                                             
⎢           3   ⎛ 2    ⎞                                                      
⎢       -l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(-sin(q₄) + cos(q₄))                                 
⎢       ──────────────────────────────────────      


M_21.T = 


⎡    4   ⎛ 2    ⎞                                          3   ⎛ 2    ⎞       
⎢-l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₀)⋅sin(q₄) + cos(q₀)⋅cos(q₄))   l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q
⎢─────────────────────────────────────────────────────  ──────────────────────
⎢                          18                                            24   
⎢                                                                             
⎢            3   ⎛ 2    ⎞                                            2   ⎛ 2  
⎢         l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(sin(q₄) + cos(q₄))                    -l⋅r ⋅ρ⋅⎝l  -
⎢         ───────────────────────────────────                    ─────────────
⎢                          24                                            12   
⎢                                                                             
⎢           3   ⎛ 2    ⎞                                                      
⎢       -l⋅r ⋅ρ⋅⎝l  - 3⎠⋅(-sin(q₄) + cos(q₄))                                 
⎢       ──────────────────────────────────────      

### Compute Internal forces 

#### 1. Transverse (Bending) Strain

In [12]:
''' Orthogonal Matricies Not Extracted to Simplify Algebra'''
R_interp = sym.simplify(H*sym.Matrix([R1,R2]))
dT = sym.simplify(H.diff(x)*sym.Matrix([R1,R2]))
kappa = sym.simplify(sym.Matrix([Axial_sym(dT*R_interp.T), '0', '0', '0']))
# display(kappa)

#### 2. Longitudinal (Axial) Strian

In [18]:
# Define Locations of Centroid 
x01 = sym.Matrix(['0','0','0'])
x02 = sym.Matrix(['1','0','0'])

# Define Newtonian Unit Vector x-dir
n1 = sym.Matrix(['1','0','0'])

# Interpolate Displacemnts
u_int = H*u
x_int = H*sym.Matrix([x01,x02])

# Derivatives w.r.t longitudinal beam coordinate
du = u_int.diff(x)
dx0 = x_int.diff(x)

# Compute axial strain
epsilon = sym.simplify(sym.Matrix.vstack(sym.Matrix(['0', '0', '0']),dx0 + du - R_interp*n1))
display(epsilon)

⎡                        0                        ⎤
⎢                                                 ⎥
⎢                        0                        ⎥
⎢                                                 ⎥
⎢                        0                        ⎥
⎢                                                 ⎥
⎢  q₁   q₅   (x - 1)⋅cos(q₀)   (x + 1)⋅cos(q₄)   1⎥
⎢- ── + ── + ─────────────── - ─────────────── + ─⎥
⎢  2    2           2                 2          2⎥
⎢                                                 ⎥
⎢    q₂   q₆   (x - 1)⋅sin(q₀)   (x + 1)⋅sin(q₄)  ⎥
⎢  - ── + ── + ─────────────── - ───────────────  ⎥
⎢    2    2           2                 2         ⎥
⎢                                                 ⎥
⎢                      q₃   q₇                    ⎥
⎢                    - ── + ──                    ⎥
⎣                      2    2                     ⎦

#### 3. Compute Internal Forces $Q_e = \frac{\partial U}{\partial e}$

In [19]:
# Transverse strain energy
Ut = 1/2*sym.integrate(E*I*kappa.dot(H.diff(x).T*H.diff(x)*kappa), (x,0,l))
# Longitudinal strain energy
Ul = 1/2*sym.integrate(E*A*epsilon.dot(H.diff(x).T*H.diff(x)*epsilon), (x,0,l))

# Compute Total Energy
U = Ul + Ut

# Compute Internal Force Vector
Qe = sym.Matrix([sym.simplify(sym.expand(sym.diff(-U,q))) for q in q])

####4. Applied and body force vector

In [20]:
# Applied forces
# Gravity body force
fg = 9.81*sym.Matrix([0,-1,0])

# Applied torque
# torque_app = sym.Matrix([0,0,tau_a])

# Compute beta
beta = sym.Matrix([sym.simplify(sym.integrate(Vr[:,j].dot(fg),('r_x',0,r),('r_y',0,r),(x,0,l)))
                   + qe for j,qe in zip(range(len(q)),Qe)])

pickle.dump( beta, open( "gebf-beta.dump", "wb" ) )

In [21]:
display(beta)

⎡    ⎛                           2                                            
⎢E⋅l⋅⎝- - -0.0208333333333333⋅A⋅l ⋅sin(q₀ - q₄) - - -0.03125⋅A⋅l⋅q₁⋅sin(q₀) + 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢     ⎛                      2                                                
⎢ E⋅l⋅⎝0.0208333333333333⋅A⋅l ⋅sin(q₀ - q₄) + 0.0312